In [17]:
import os
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import LineString
from IPython.display import IFrame

# Crear carpeta de salida si no existe
os.makedirs("../output", exist_ok=True)

# Cargar archivos con normalización estricta
df_result = pd.read_csv("../output/resultados_validacion.csv")
df_pois = pd.read_csv("../data/POIs/POI_4815078.csv")

# Estandariza nombres de columnas (mayúsculas y sin espacios)
df_result.columns = [col.strip().upper() for col in df_result.columns]
df_pois.columns = [col.strip().upper() for col in df_pois.columns]

# Elimina columnas duplicadas (ahora que están unificadas en mayúsculas)
df_result = df_result.loc[:, ~df_result.columns.duplicated()]
df_pois = df_pois.loc[:, ~df_pois.columns.duplicated()]

# Ahora sí puedes hacer el merge sin errores
df = pd.merge(df_result, df_pois, on="POI_ID", how="left")


dupes = [col for col in df_result.columns if df_result.columns.tolist().count(col) > 1]
print("Columnas duplicadas en df_result:", dupes)

# Diccionario de colores por tipo de acción
color_dict = {
    "eliminar": "red",
    "corregir lado (POI_ST_SD)": "orange",
    "cambiar MULTIDIGIT a N": "blue",
    "marcar como excepción": "green"
}

# Calcular coordenadas de cada POI según interpolación
points = []
for _, row in df.iterrows():
    try:
        link_id = row["LINK_ID"]
        perc = float(row["PERCFRREF"])

        match = gdf_calles[gdf_calles["link_id"] == link_id]
        if match.empty:
            continue

        geom = match.iloc[0].geometry
        if not isinstance(geom, LineString):
            continue

        point = geom.interpolate(perc * geom.length)
        lat, lon = point.y, point.x

        points.append({
            "POI_ID": row["POI_ID"],
            "LAT": lat,
            "LON": lon,
            "ACCION": row.get("ACCION", "sin acción"),
            "COMENTARIO": row.get("COMENTARIO", "sin comentario")
        })

    except Exception as e:
        print(f"Error en fila: {row.get('POI_ID')}, error: {e}")
        continue

# Crear DataFrame de coordenadas
df_coords = pd.DataFrame(points)

# Crear mapa base
mapa = folium.Map(location=[df_coords["LAT"].mean(), df_coords["LON"].mean()], zoom_start=15)

# Ajustar el encuadre al área de los puntos
bounds = [[df_coords["LAT"].min(), df_coords["LON"].min()],
          [df_coords["LAT"].max(), df_coords["LON"].max()]]
mapa.fit_bounds(bounds)

# Agregar marcadores al mapa
for _, row in df_coords.iterrows():
    color = color_dict.get(row["ACCION"], "gray")
    popup = f"<b>{row['ACCION']}</b><br>{row['COMENTARIO']}"
    folium.CircleMarker(
        location=[row["LAT"], row["LON"]],
        radius=4,
        color=color,
        fill=True,
        fill_opacity=0.75,
        popup=folium.Popup(popup, max_width=300)
    ).add_to(mapa)

# Agregar leyenda visual
legend_html = """
<div style='position: fixed; bottom: 50px; left: 50px; width: 230px; height: 120px;
     border:2px solid grey; z-index:9999; font-size:14px; background-color:white;
     padding: 10px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);'>
<b>🗺️ Leyenda:</b><br>
🔴 Eliminar<br>
🟠 Corregir lado<br>
🔵 Cambiar MULTIDIGIT<br>
🟢 Excepción válida
</div>
"""
mapa.get_root().html.add_child(folium.Element(legend_html))

# Guardar y mostrar
map_path = "../output/mapa_resultado.html"
mapa.save(map_path)
IFrame(map_path, width=900, height=500)




Columnas duplicadas en df_result: []


In [16]:
import folium
import geopandas as gpd

# 1. Cargar el GeoDataFrame de calles si no lo tienes ya

calles = gpd.read_file("../data/STREETS_NAV/SREETS_NAV_4815075.geojson")
calles.columns = [col.lower() for col in calles.columns]

# 2. LINK_ID del POI frente a la Escuela Miguel Hidalgo
link_id = 702696670
calle = calles[calles["link_id"] == link_id]
geom = calle.iloc[0].geometry

# 4. Coordenadas del LineString
coords = list(geom.coords)
start = coords[0]
end = coords[-1]

print("Punto de inicio:", start)
print("Punto final:", end)

# 5. Mostrar dirección de la calle en mapa
m = folium.Map(location=[(start[1] + end[1]) / 2, (start[0] + end[0]) / 2], zoom_start=19)

# Línea principal
folium.PolyLine(locations=[(pt[1], pt[0]) for pt in coords], color="blue").add_to(m)

# Flechitas de inicio y fin
folium.Marker(location=(start[1], start[0]), popup="Inicio (REF NODE)", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=(end[1], end[0]), popup="Fin", icon=folium.Icon(color="red")).add_to(m)

m


Punto de inicio: (-99.6262, 19.2705)
Punto final: (-99.62733, 19.27055)


In [7]:
import folium
import geopandas as gpd

# 1. Cargar el GeoDataFrame de calles si no lo tienes ya
calles = gpd.read_file("../data/STREETS_NAV/SREETS_NAV_4815075.geojson")
calles.columns = [col.lower() for col in calles.columns]

# 2. LINK_ID del segmento que quieras analizar
link_id = 780258077 # reemplázalo con el ID que quieras analizar

# 3. Obtener la geometría
calle = calles[calles["link_id"] == link_id]
geom = calle.iloc[0].geometry

# 4. Coordenadas del LineString
coords = list(geom.coords)
start = coords[0]
end = coords[-1]

print("Punto de inicio:", start)
print("Punto final:", end)
print("¿MULTIDIGIT?", calle.iloc[0]["multidigit"])

# 5. Mostrar dirección de la calle en mapa
m = folium.Map(location=[(start[1] + end[1]) / 2, (start[0] + end[0]) / 2], zoom_start=19)

# Línea principal
folium.PolyLine(locations=[(pt[1], pt[0]) for pt in coords], color="blue").add_to(m)

# Flechitas de inicio y fin
folium.Marker(location=(start[1], start[0]), popup="Inicio (REF NODE)", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(location=(end[1], end[0]), popup="Fin", icon=folium.Icon(color="red")).add_to(m)

m



Punto de inicio: (-99.62613, 19.26974)
Punto final: (-99.62585, 19.27138)
¿MULTIDIGIT? N


In [4]:
import sys
import os

# Agrega la ruta raíz de tu proyecto al path
sys.path.append(os.path.abspath(".."))  # Sube un nivel hasta .heresolution
# Ahora sí, importa
from src.here_api import buscar_lugar, imagen_satelital

pos = buscar_lugar("Catedral de Toluca")
print(pos)  # {'lat': 19.287, 'lng': -99.655} (por ejemplo)

# Descargar imagen satelital
path = imagen_satelital(pos['lat'], pos['lng'])
print("Imagen guardada en:", path)


Imagen guardada en: {'error': 'Error 429', 'message': '{"error":"Too Many Requests","error_description":"Rate limit for this service has been reached"}\n'}
